In [1]:
from pyace import *
import pandas as pd
import numpy as np
from tqdm import tqdm  # Progress bar

# Load ACE calculator
folder = 'FN-all'
calc = PyACECalculator(folder + "/ensemble_potential_0.yaml")
calc.set_active_set(folder + "/ensemble_potential_0.asi")

# Load atomic configurations
df = pd.read_pickle("group2.pckl.gzip", compression="gzip")
atoms_list = df["ase_atoms"]

# Initialize lists to store results
all_gammas = []       # Will store all gamma values (flattened later)
all_gamma_max = []    # Stores max gamma per structure

# Compute gamma and gamma_max for each structure
for atoms in tqdm(atoms_list, desc="Processing structures"):
    atoms.set_calculator(calc)
    atoms.get_potential_energy()  # Triggers gamma calculation
    gamma = calc.results["gamma"]
    all_gammas.extend(gamma)      # Flatten into 1D list
    all_gamma_max.append(np.max(gamma))

# Convert to numpy arrays
gamma_flattened = np.array(all_gammas)
gamma_max_array = np.array(all_gamma_max)

# Save flattened gamma values (one value per line)
np.savetxt('gamma2/'+folder+"-gamma.txt", gamma_flattened, fmt="%.8f")

# Save gamma_max values (one per structure)
np.savetxt('gamma2/'+folder+"-gamma-max.txt", gamma_max_array, fmt="%.8f")

print("Results saved:")
print(f"- gamma.txt: Flattened gamma values (shape: {gamma_flattened.shape})")
print(f"- gamma_max.txt: Max gamma per structure (shape: {gamma_max_array.shape})")

Processing structures:   0%|                                                                                                        | 0/1383 [00:00<?, ?it/s]/tmp/ipykernel_682/2867385622.py:21: FutureWarning: Please use atoms.calc = calc
  atoms.set_calculator(calc)
Processing structures: 100%|█████████████████████████████████████████████████████████████████████████████████████████████| 1383/1383 [03:48<00:00,  6.06it/s]


Results saved:
- gamma.txt: Flattened gamma values (shape: (210679,))
- gamma_max.txt: Max gamma per structure (shape: (1383,))
